# GeoSpatial Data Project

In [1]:
import src.funciones_llamada_api as fla
import os
from dotenv import load_dotenv

In [2]:
# Librerias
import pandas as pd

In [3]:
# Librerias visualizacion json
import pyjsonviewer

### asignamos los token de la carpeta .env a unas varibles que creamos

In [4]:
load_dotenv()

# asignamos los token de la carpeta .env a unas varibles que creamos
token = os.getenv("Client_Id")
token2 = os.getenv("Client_Secret")

#### con geocode buscamos la longitud y la latitud de la direccion que le demos

In [5]:
dir_madrid = "Plaza de Colón, Madrid"
dir_alcobendas = "Heron Diversia, Avenida de Bruselas, Alcobendas"
dir_rozas = "Heron City Las Rozas, Calle Camilo José Cela, Las Rozas de Madrid"

# Llamamos a la funcion que nos pasa las coordenadas con geocode
madrid = fla.geocode_coord(dir_madrid)
alcobendas = fla.geocode_coord(dir_alcobendas)
rozas = fla.geocode_coord(dir_rozas)

In [5]:
madrid = {'type': 'Point', 'coordinates': ['40.42562', '-3.69104']}

##### funcion generar_json, obtener_datos y json_filtrado

In [6]:
resp = fla.generar_json(madrid, "vegan", token, token2)

In [7]:
datos_finales = fla.obtener_datos(resp)

In [8]:
type(datos_finales)

list

In [13]:
#from functools import reduce
#import operator

In [9]:
datos_finales

[{'id': '59bd2433805e3f5982c09476',
  'name': "Bunny's Deli",
  'contact': {},
  'location': {'address': 'Calle San Gregorio 17',
   'lat': 40.423935,
   'lng': -3.6971583,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.423935,
     'lng': -3.6971583}],
   'distance': 551,
   'postalCode': '28004',
   'cc': 'ES',
   'neighborhood': 'Chueca',
   'city': 'Madrid',
   'state': 'Madrid',
   'country': 'España',
   'formattedAddress': ['Calle San Gregorio 17',
    '28004 Madrid Madrid',
    'España']},
  'categories': [{'id': '4bf58dd8d48988d1d3941735',
    'name': 'Vegetarian / Vegan Restaurant',
    'pluralName': 'Vegetarian / Vegan Restaurants',
    'shortName': 'Vegetarian / Vegan',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
     'suffix': '.png'},
    'primary': True}],
  'verified': True,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'checkinsCount': 0,
   'visitsCount': 0},
  'beenHere': {'count': 0,
   'lastCheckinExpiredAt': 0,


In [10]:
jsonfinal = fla.json_filtrado(datos_finales)

In [11]:
jsonfinal

[{'nombre': "Bunny's Deli",
  'latitud': 40.423935,
  'longitud': -3.6971583,
  'location': {'type': 'Point', 'coordinates': [-3.6971583, 40.423935]}},
 {'nombre': 'Roots',
  'latitud': 40.424572,
  'longitud': -3.6955872,
  'location': {'type': 'Point', 'coordinates': [-3.6955872, 40.424572]}},
 {'nombre': 'Ohanasana',
  'latitud': 40.423075,
  'longitud': -3.695536,
  'location': {'type': 'Point', 'coordinates': [-3.695536, 40.423075]}},
 {'nombre': 'La Galette 2 - Casa Escondida',
  'latitud': 40.423449262141645,
  'longitud': -3.69254024923133,
  'location': {'type': 'Point',
   'coordinates': [-3.69254024923133, 40.423449262141645]}},
 {'nombre': 'Restaurante Vegaviana',
  'latitud': 40.42327219098249,
  'longitud': -3.69815135043315,
  'location': {'type': 'Point',
   'coordinates': [-3.69815135043315, 40.42327219098249]}},
 {'nombre': 'La Hummuseria',
  'latitud': 40.423474938716694,
  'longitud': -3.700036508653173,
  'location': {'type': 'Point',
   'coordinates': [-3.70003650

In [12]:
jsonfinal[0]

{'nombre': "Bunny's Deli",
 'latitud': 40.423935,
 'longitud': -3.6971583,
 'location': {'type': 'Point', 'coordinates': [-3.6971583, 40.423935]}}

#### exportamos el diccionaro a un json

In [32]:
# visualizar json .view_data(json_file="ruta")
import json
with open('prueba.json', 'w') as f:
    json.dump(unjsonnuevo, f)